# Detect Separation Between Two Phenotypes in TCGA, GTEx, and TARGET Data

**Gregory Way, 2019**

Perform a t-test between two distinct phenotypes. In TCGA and GTEx data, we perform a t-test on males and females while in TARGET data, we test MYCN amplified vs. MYCN not amplified neuroblastoma tumors. We track the t-test p values across k dimensions and algorithms to isolate the features that best distinguishes the two groups.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, ttest_rel
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append("../8.gtex-interpret")
from scripts.utils import (
    load_weight_matrix,
    apply_signature,
    load_enrichment_results,
    extract_feature,
)

sys.path.append("../9.tcga-classify/")
from scripts.tcga_util import build_feature_dictionary

In [2]:
def ttest_difference(feature_series, group_a_ids, group_b_ids):
    """
    To be applied to a pandas dataframe by column
    """
    feature_name = feature_series.name
    feature_algorithm, feature_num = feature_name.split('_')
    
    a_activation = feature_series[feature_series.index.isin(group_a_ids)]
    b_activation = feature_series[feature_series.index.isin(group_b_ids)]
    
    # Perform t-test on two groups
    t_stat, t_p = ttest_ind(a_activation, b_activation, equal_var=False)
    
    return([t_stat, t_p, feature_algorithm, feature_num])


def get_ttest_results(z_matrix_dict, group_a_ids, group_b_ids, train_or_test='test'):
    """
    Loop through z matrix performing t-test using the compressed feature scores.
    Output full t-test results
    """
    
    # Perform t-test for all compressed features
    full_results = []
    for signal in z_matrix_dict.keys():
        for z_dim in z_matrix_dict[signal].keys():
            for seed in z_matrix_dict[signal][z_dim].keys():
                if train_or_test == "both":
                    z_train_df = z_matrix_dict[signal][z_dim][seed]["train"]
                    z_test_df = z_matrix_dict[signal][z_dim][seed]["test"]
                    z_df = pd.concat([z_train_df, z_test_df])
                else:
                    z_df = z_matrix_dict[signal][z_dim][seed][train_or_test]

                result_df = pd.DataFrame(z_df.apply(lambda x:
                                                    ttest_difference(feature_series=x,
                                                                     group_a_ids=group_a_ids,
                                                                     group_b_ids=group_b_ids)),
                                         columns = ['result'])

                result_df = (
                    pd.DataFrame(result_df.result.values.tolist(),
                                 columns=['t_stat', 't_p', 'algorithm', 'feature_num'])
                ).fillna(1)

                result_df = result_df.assign(
                    z_dim=z_dim,
                    signal=signal,
                    seed=seed
                )
                full_results.append(result_df)
    
    full_results_df = pd.concat(full_results)
    full_results_df = full_results_df.assign(neg_log_p=-np.log10(full_results_df.t_p + 1e-300))
    full_results_df = full_results_df.sort_values(by='neg_log_p', ascending=False)
    return full_results_df

## 1. GTEx Sex Analysis

In [3]:
# Load GTEx phenotype data
file = os.path.join("..", "0.expression-download", "download", "GTEx_v7_Annotations_SubjectPhenotypesDS.txt")
gtex_pheno_df = pd.read_table(file)
gtex_pheno_df.head()

,SUBJID,SEX,AGE,DTHHRDY
0,GTEX-1117F,2,60-69,4.0
1,GTEX-111CU,1,50-59,0.0
2,GTEX-111FC,1,60-69,1.0
3,GTEX-111VG,1,60-69,3.0
4,GTEX-111YS,1,60-69,0.0


In [4]:
# Load GTEx Sample Attribute Data
file = os.path.join("..", "0.expression-download", "download", "GTEx_v7_Annotations_SampleAttributesDS.txt")
gtex_sample_df = pd.read_table(file)

print(gtex_sample_df.shape)
gtex_sample_df.head(2)

(15598, 63)


,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN,SMTS,SMTSD,SMUBRID,SMTSISCH,SMTSPAX,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
0,GTEX-1117F-0003-SM-58Q7G,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GTEX-1117F-0003-SM-5DWSB,NaN,B1,NaN,NaN,Blood,Whole Blood,0013756,1188.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Load compressed matrix
gtex_z_matrix_dict = build_feature_dictionary(dataset="GTEX",
                                              load_data=True,
                                              store_train_test='test')

In [6]:
# Extract male and female ids from the dataset using one matrix as an example
# (All matrices are aligned with the same IDs)
example_matrix_df = gtex_z_matrix_dict['signal']['8']['451283']['test']

patient_id_df = pd.concat(
    [
    pd.DataFrame(["{}-{}".format(x[0], x[1]) for x in example_matrix_df.index.str.split('-')],
                 columns=['patient_id'])
        .merge(gtex_pheno_df,
               how='left',
               left_on='patient_id',
               right_on='SUBJID'),
    pd.DataFrame(example_matrix_df.index)
    ],
    axis='columns'
)

patient_id_df = (
    patient_id_df.merge(
        gtex_sample_df,
        left_on="sample_id",
        right_on="SAMPID",
        how="left"
    )
)

print(patient_id_df.shape)
patient_id_df.head(3)

(1169, 69)


,patient_id,SUBJID,SEX,AGE,DTHHRDY,sample_id,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,...,SME1ANTI,SMSPLTRD,SMBSMMRT,SME1SNSE,SME1PCTS,SMRRNART,SME1MPRT,SMNUM5CD,SMDPMPRT,SME2PCTS
0,GTEX-ZTX8,GTEX-ZTX8,1,20-29,0.0,GTEX-ZTX8-1126-SM-51MRM,GTEX-ZTX8-1126-SM-51MRM,1.0,B1,2 pieces,...,21116618.0,15670572.0,0.004129,21302446.0,50.219040,0.014548,0.993089,NaN,0.0,49.90624
1,GTEX-Y3IK,GTEX-Y3IK,2,50-59,0.0,GTEX-Y3IK-2426-SM-4WWDU,GTEX-Y3IK-2426-SM-4WWDU,0.0,B1,"6 pieces, up to 9x2mm. Well trimmed, but with ...",...,21949011.0,19389955.0,0.003998,22107996.0,50.180430,0.005525,0.985788,NaN,0.0,50.02296
2,GTEX-X62O,GTEX-X62O,1,50-59,0.0,GTEX-X62O-0826-SM-46MW8,GTEX-X62O-0826-SM-46MW8,2.0,C1,"2 pieces, 8x7 & 7x7mm;",...,12511646.0,10713700.0,0.002388,12536978.0,50.050564,0.009804,0.993809,NaN,0.0,50.04157


In [7]:
# Get gender ratio across samples
gender_type_ratio = pd.crosstab(patient_id_df.SMTS, patient_id_df.SEX)
gender_type_ratio = (
    gender_type_ratio
    .assign(ratio=((gender_type_ratio.loc[:, 2] + 1) / (gender_type_ratio.loc[:, 1] + 1)))
)

fudge_factor = 0.5
balanced= (
    gender_type_ratio
    .loc[
        (gender_type_ratio.ratio < (1 + fudge_factor)) &
        (gender_type_ratio.ratio > (1 - fudge_factor)),
        :]
)

out_file = os.path.join("results", "balanced_gtex_tissues.tsv")
balanced.to_csv(out_file, sep='\t')
        
balanced

SEX,1,2,ratio
SMTS,,,
Adrenal Gland,10,9,0.909091
Blood Vessel,55,36,0.660714
Brain,111,56,0.508929
Breast,19,10,0.550000
Colon,31,19,0.625000
Esophagus,61,41,0.677419
Heart,40,20,0.512195
Lung,27,16,0.607143
Muscle,33,23,0.705882


In [8]:
balanced_tissue_types = balanced.index.tolist()

gtex_males = (
    patient_id_df
    .query("SEX == 1")
    .query("SMTS in @balanced_tissue_types")
    .sample_id.tolist()
)
gtex_females = (
    patient_id_df
    .query("SEX == 2")
    .query("SMTS in @balanced_tissue_types")
    .sample_id.tolist()
)

print(len(gtex_males))
print(len(gtex_females))

500
304


In [9]:
# Perform t-test for all compressed features
gtex_full_results_df = get_ttest_results(z_matrix_dict=gtex_z_matrix_dict,
                                         group_a_ids=gtex_males,
                                         group_b_ids=gtex_females,
                                         train_or_test="test")

In [10]:
# Output results
file = os.path.join("results", "sex_separation_gtex_t_test.tsv")
gtex_full_results_df.to_csv(file, sep='\t', index=False)

print(gtex_full_results_df.shape)
gtex_full_results_df.head()

(61700, 8)


,t_stat,t_p,algorithm,feature_num,z_dim,signal,seed,neg_log_p
511,44.507291,7.302909e-176,nmf,111,200,signal,451283,175.136504
411,43.596212,2.655225e-172,nmf,111,150,signal,486191,171.575899
361,43.448300,1.021953e-171,nmf,111,125,signal,486191,170.990569
411,43.297948,4.074371e-171,nmf,111,150,signal,165158,170.389939
411,42.894468,1.614220e-169,nmf,111,150,signal,978124,168.792037


## 2. TCGA Sex Analysis

In [11]:
# Load TCGA phenotype data
file = os.path.join("..", "0.expression-download", "download", "TCGA-CDR-SupplementalTableS1.xlsx")
tcga_pheno_df = pd.read_excel(file)

tcga_pheno_df.head()

,bcr_patient_barcode,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,initial_pathologic_dx_year,...,residual_tumor,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction
1,TCGA-OR-A5J1,ACC,58.0,MALE,WHITE,Stage II,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,...,NaN,1.0,1355.0,1.0,1355.0,1.0,754.0,1.0,754.0,NaN
2,TCGA-OR-A5J2,ACC,44.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2004.0,...,NaN,1.0,1677.0,1.0,1677.0,NaN,NaN,1.0,289.0,NaN
3,TCGA-OR-A5J3,ACC,23.0,FEMALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2008.0,...,NaN,0.0,2091.0,0.0,2091.0,1.0,53.0,1.0,53.0,NaN
4,TCGA-OR-A5J4,ACC,23.0,FEMALE,WHITE,Stage IV,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,...,NaN,1.0,423.0,1.0,423.0,NaN,NaN,1.0,126.0,NaN
5,TCGA-OR-A5J5,ACC,30.0,MALE,WHITE,Stage III,[Not Applicable],Adrenocortical carcinoma- Usual Type,[Not Available],2000.0,...,NaN,1.0,365.0,1.0,365.0,NaN,NaN,1.0,50.0,NaN


In [12]:
tcga_z_matrix_dict = build_feature_dictionary(dataset="TCGA",
                                              load_data=True,
                                              store_train_test='test')

In [13]:
# Extract male and female ids from the dataset
example_matrix_df = tcga_z_matrix_dict['signal']['2']['451283']['test']

patient_id_df = pd.concat(
    [
    pd.DataFrame(["{}-{}-{}".format(x[0], x[1], x[2]) for x in example_matrix_df.index.str.split('-')],
                 columns=['patient_id'])
        .merge(tcga_pheno_df,
               how='left',
               left_on='patient_id',
               right_on='bcr_patient_barcode'),
    pd.DataFrame(example_matrix_df.index)
    ],
    axis='columns'
)

print(patient_id_df.shape)
patient_id_df.head()

(1106, 35)


,patient_id,bcr_patient_barcode,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction,sample_id
0,TCGA-CN-5365,TCGA-CN-5365,HNSC,38.0,MALE,WHITE,Stage IVB,Stage IVC,Head & Neck Squamous Cell Carcinoma,G2,...,1.0,351.0,1.0,351.0,NaN,NaN,1.0,351.0,NaN,TCGA-CN-5365-01
1,TCGA-LP-A7HU,TCGA-LP-A7HU,CESC,53.0,FEMALE,ASIAN,[Not Available],Stage II,Endocervical Type of Adenocarcinoma,G3,...,0.0,406.0,0.0,406.0,0.0,406.0,0.0,406.0,NaN,TCGA-LP-A7HU-01
2,TCGA-22-5491,TCGA-22-5491,LUSC,74.0,MALE,WHITE,Stage IA,[Not Applicable],Lung Squamous Cell Carcinoma,[Not Available],...,1.0,1713.0,NaN,1713.0,NaN,NaN,0.0,1713.0,NaN,TCGA-22-5491-11
3,TCGA-CS-6667,TCGA-CS-6667,LGG,39.0,FEMALE,WHITE,[Not Available],[Not Available],Astrocytoma,G2,...,0.0,1469.0,0.0,1469.0,NaN,NaN,0.0,1469.0,NaN,TCGA-CS-6667-01
4,TCGA-20-1684,TCGA-20-1684,OV,51.0,FEMALE,WHITE,[Not Applicable],Stage IIIC,Serous Cystadenocarcinoma,G3,...,0.0,581.0,0.0,581.0,0.0,581.0,0.0,581.0,NaN,TCGA-20-1684-01


In [14]:
gender_type_ratio = pd.crosstab(patient_id_df.loc[:, "type"], patient_id_df.gender)
gender_type_ratio = (
    gender_type_ratio
    .assign(ratio=((gender_type_ratio.FEMALE + 1) / (gender_type_ratio.MALE + 1)))
)

fudge_factor = 0.5
balanced= (
    gender_type_ratio
    .loc[
        (gender_type_ratio.ratio < (1 + fudge_factor)) &
        (gender_type_ratio.ratio > (1 - fudge_factor)),
        :]
)

out_file = os.path.join("results", "balanced_tcga_tissues.tsv")
balanced.to_csv(out_file, sep='\t')

balanced

gender,FEMALE,MALE,ratio
type,,,
COAD,28,21,1.318182
DLBC,3,2,1.333333
GBM,6,10,0.636364
HNSC,19,37,0.526316
KICH,5,5,1.000000
KIRC,21,39,0.550000
KIRP,11,22,0.521739
LAML,8,9,0.900000
LGG,26,27,0.964286


In [15]:
balanced_cancer_types = balanced.index.tolist()

tcga_males = (
    patient_id_df
    .query("gender == 'MALE'")
    .query("type in @balanced_cancer_types")
    .sample_id.tolist()
)
tcga_females = (
    patient_id_df
    .query("gender == 'FEMALE'")
    .query("type in @balanced_cancer_types")
    .sample_id.tolist()
)

print(len(tcga_males))
print(len(tcga_females))

267
218


In [16]:
# Perform t-test for all compressed features
tcga_full_results_df = get_ttest_results(z_matrix_dict=tcga_z_matrix_dict,
                                         group_a_ids=tcga_males,
                                         group_b_ids=tcga_females,
                                         train_or_test="test")

In [17]:
# Output results
file = os.path.join("results", "sex_separation_tcga_t_test.tsv")
tcga_full_results_df.to_csv(file, sep='\t', index=False)

print(tcga_full_results_df.shape)
tcga_full_results_df.head()

(61700, 8)


,t_stat,t_p,algorithm,feature_num,z_dim,signal,seed,neg_log_p
143,4.864065,0.000002,ica,53,90,signal,165158,5.798404
134,-4.857106,0.000002,ica,44,90,signal,451283,5.779352
105,-4.797379,0.000002,ica,15,90,signal,908341,5.657098
104,4.766643,0.000003,ica,14,90,signal,486191,5.593012
132,4.621749,0.000005,ica,42,90,signal,978124,5.299861


## 3. TARGET NBL MYCN Status Analysis

In [18]:
# Load TARGET phenotype data
file = os.path.join("..", "0.expression-download", "data", "2017-09-30-TARGET update harmonized.txt")
nbl_pheno_df = pd.read_table(file)
nbl_pheno_df.head()

,usi,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,Year of Diagnosis,...,Histology,Grade,MKI,Diagnostic Category,ICDO,ICDO Description,COG Risk Group,Site Relapse,Comment,target_update
0,PAAPFA,Male,White,Not Hispanic or Latino,1762,Event,444.0,Dead,487.0,1986.0,...,Unfavorable,Unknown,Unknown,Unknown,Unknown,Unknown,High Risk,NaN,NaN,old
1,PACLJN,Male,White,Not Hispanic or Latino,1475,Censored,5553.0,Alive,5553.0,1986.0,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,High Risk,NaN,NaN,old
2,PACPJG,Female,White,Not Hispanic or Latino,760,Unknown,NaN,Unknown,NaN,1987.0,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,NaN,NaN,old
3,PACRYY,Male,Unknown,Hispanic or Latino,1314,Censored,5296.0,Alive,5296.0,1987.0,...,Unfavorable,Unknown,Unknown,Unknown,Unknown,Lge Right retroperitoneal mass extending thru ...,High Risk,NaN,NaN,old
4,PACRZM,Male,White,Not Hispanic or Latino,3686,Event,922.0,Dead,922.0,1987.0,...,Unfavorable,Unknown,Unknown,Unknown,Unknown,Unknown,High Risk,NaN,NaN,old


In [19]:
# Load TARGET matrices
target_z_matrix_dict = build_feature_dictionary(dataset="TARGET",
                                                load_data=True,
                                                store_train_test='both')

In [20]:
# Extract amplified and not amplified samples from the dataset
example_train_matrix_df = target_z_matrix_dict['signal']['8']['451283']['train']
example_test_matrix_df = target_z_matrix_dict['signal']['8']['451283']['test']
example_matrix_df = pd.concat([example_train_matrix_df, example_test_matrix_df])

patient_id_df = (
    pd.concat(
        [
            pd.DataFrame([x[2] for x in example_matrix_df.index.str.split('-')],
                         columns=['patient_id'])
            .merge(nbl_pheno_df,
                   how='left',
                   left_on='patient_id',
                   right_on='usi'),
            pd.DataFrame(example_matrix_df.index)
        ],
        axis='columns'
    )
    .dropna(subset=['usi'])
)

patient_id_df = patient_id_df.loc[patient_id_df["Diagnostic Category"] == "Neuroblastoma", :]

mycn_amp = patient_id_df.loc[patient_id_df["MYCN status"] == "Amplified", "sample_id"].tolist()
mycn_nonamp = patient_id_df.loc[patient_id_df["MYCN status"] == "Not Amplified", "sample_id"].tolist()

print(patient_id_df.shape)
patient_id_df.head()

(130, 28)


,patient_id,usi,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,...,Grade,MKI,Diagnostic Category,ICDO,ICDO Description,COG Risk Group,Site Relapse,Comment,target_update,sample_id
0,PARSBI,PARSBI,Female,White,Not Hispanic or Latino,2390.0,Relapse,1377.0,Dead,1743.0,...,Undifferentiated or Poorly Differentiated,Low,Neuroblastoma,C76.2,"Abdomen, NOS Abdominal wall, NOS Intra-abdom...",High Risk,Other metastatic sites,NaN,old,TARGET-30-PARSBI-01
11,PARNNC,PARNNC,Female,White,Not Hispanic or Latino,41.0,Relapse,735.0,Alive,2979.0,...,Undifferentiated or Poorly Differentiated,Low,Neuroblastoma,C48.0,Retroperitoneum\n\nPeriadrenal tissue\n\nPerin...,Low Risk,Primary site;; Other metastatic sites,NaN,old,TARGET-30-PARNNC-01
18,PASVRU,PASVRU,Male,White,Hispanic or Latino,631.0,Event,254.0,Dead,440.0,...,Undifferentiated or Poorly Differentiated,High,Neuroblastoma,C48.0,Retroperitoneum Periadrenal tissue Perinephr...,High Risk,NaN,NaN,old,TARGET-30-PASVRU-01
22,PALTEG,PALTEG,Male,White,Not Hispanic or Latino,1367.0,Relapse,523.0,Dead,551.0,...,Undifferentiated or Poorly Differentiated,Intermediate,Neuroblastoma,C76.2,"Abdomen, NOS Abdominal wall, NOS Intra-abdom...",High Risk,Other Metastatic Sites,NaN,old,TARGET-30-PALTEG-01
24,PASJZC,PASJZC,Female,White,Not Hispanic or Latino,74.0,Relapse,367.0,Alive,2631.0,...,Undifferentiated or Poorly Differentiated,Low,Neuroblastoma,C22.0,"Liver\n\nHepatic, NOS",High Risk,Primary site,NaN,old,TARGET-30-PASJZC-01


In [21]:
# Perform t-test for all compressed features
target_full_results_df = get_ttest_results(z_matrix_dict=target_z_matrix_dict,
                                           group_a_ids=mycn_amp,
                                           group_b_ids=mycn_nonamp,
                                           train_or_test='both')

In [22]:
file = os.path.join("results", "nbl_mycn_separation_target_t_test.tsv")
target_full_results_df.to_csv(file, sep='\t', index=False)

print(target_full_results_df.shape)
target_full_results_df.head()

(61700, 8)


,t_stat,t_p,algorithm,feature_num,z_dim,signal,seed,neg_log_p
162,-18.539478,6.613869e-38,vae,12,50,signal,451283,37.179544
488,-17.652445,7.166732e-36,dae,88,100,signal,451283,35.144679
155,-17.324506,9.638913e-35,vae,5,50,signal,978124,34.015972
545,-16.472087,6.465500e-32,vae,95,150,signal,908341,31.189398
72,-16.650011,9.533733e-31,nmf,2,35,signal,486191,30.020737
